In [5]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pygame
import random
import math

# Initialize Pygame
pygame.init()

# Screen Dimensions
WIDTH = 800
HEIGHT = 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Cosmic Defender: Stellar Warfare")

# Colors
BLACK = (10, 10, 20)
WHITE = (255, 255, 255)
BLUE = (66, 135, 245)
RED = (255, 77, 77)
GREEN = (0, 255, 0)
YELLOW = (255, 255, 0)
PURPLE = (128, 0, 128)
DARK_BLUE = (0, 0, 100)

# Star Background with Parallax Effect
class StarField:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.stars = []
        
        # Generate stars with varying speeds
        for _ in range(300):  # Increased the number of stars
            x = random.randrange(0, width)
            y = random.randrange(0, height)
            speed = random.uniform(0.1, 2)
            size = random.randint(1, 3)
            self.stars.append([x, y, speed, size])
    
    def update(self):
        for star in self.stars:
            # Move star down with different speeds
            star[1] += star[2]
            
            if star[1] > self.height:
                star[0] = random.randrange(0, self.width)
                star[1] = 0
                star[2] = random.uniform(0.1, 2)
                star[3] = random.randint(1, 3)
    
    def draw(self, surface):
        for star in self.stars:
            brightness = int(100 + star[2] * 50)
            color = (min(brightness, 255), 
                     min(brightness, 255), 
                     min(brightness, 255))
            pygame.draw.circle(surface, color, (int(star[0]), int(star[1])), star[3])

# Player Class with Shading and Gradient Effect
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

        # Create a more dynamic player ship with a gradient effect
        self.original_image = pygame.Surface((60, 80), pygame.SRCALPHA)
        
        # Draw body with gradient
        for i in range(0, 60, 5):
            color = (66, 135, 245) if i < 30 else (66, 100, 245)  # Darker as we move down
            pygame.draw.polygon(self.original_image, color, [(i, 0), (0, 80), (60, 80), (60 - i, 0)])
        
        # Add engine glow with gradient effect
        glow_surface = pygame.Surface((60, 30), pygame.SRCALPHA)
        glow_color = (0, 255, 255, 80)  # Lighter cyan glow
        pygame.draw.ellipse(glow_surface, glow_color, glow_surface.get_rect())
        self.original_image.blit(glow_surface, (0, 70))

        # Weapon mounts
        pygame.draw.rect(self.original_image, YELLOW, (15, 60, 10, 20), 2)
        pygame.draw.rect(self.original_image, YELLOW, (35, 60, 10, 20), 2)
        
        self.image = self.original_image
        self.rect = self.image.get_rect()
        self.rect.centerx = WIDTH // 2
        self.rect.bottom = HEIGHT - 10
        self.speed = 5
        self.health = 100  # Player's health
        self.weapon_level = 1
        self.bullet_count = 2  # Starting with 2 bullets
        
    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and self.rect.left > 0:
            self.rect.x -= self.speed
        if keys[pygame.K_RIGHT] and self.rect.right < WIDTH:
            self.rect.x += self.speed

    def lose_health(self, damage):
        self.health -= damage
        if self.health < 0:
            self.health = 0

    def increase_bullets(self):
        self.bullet_count += 1

# Enhanced Enemy Class with Rotation and Detailing
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        
        # Enemy shape with more dynamic features
        self.original_image = pygame.Surface((40, 50), pygame.SRCALPHA)
        
        # Alien-like ship body
        pygame.draw.polygon(self.original_image, RED, 
            [(20, 0), (0, 50), (40, 50)])
        
        # Add detailing such as wing lines
        pygame.draw.line(self.original_image, PURPLE, (5, 40), (0, 50), 3)
        pygame.draw.line(self.original_image, PURPLE, (35, 40), (40, 50), 3)
        
        # Add rotating effect to the enemy
        self.image = self.original_image
        self.rect = self.image.get_rect()
        self.rect.x = random.randrange(WIDTH - self.rect.width)
        self.rect.y = random.randrange(-100, -40)
        self.speedy = random.randrange(1, 4)
        self.speedx = random.randrange(-1, 2)
        self.rotation_angle = random.uniform(0, 360)
        
    def update(self):
        self.rect.y += self.speedy
        self.rect.x += self.speedx
        
        # Rotate the enemy ship for some dynamic visual effects
        self.rotation_angle += 2  # Rotate by 2 degrees per frame
        if self.rotation_angle >= 360:
            self.rotation_angle = 0
        self.image = pygame.transform.rotate(self.original_image, self.rotation_angle)
        self.rect = self.image.get_rect(center=self.rect.center)

        # Respawn if off screen
        if self.rect.top > HEIGHT:
            self.rect.x = random.randrange(WIDTH - self.rect.width)
            self.rect.y = random.randrange(-100, -40)
            self.speedy = random.randrange(1, 4)
            self.speedx = random.randrange(-1, 2)

# Laser with Glowing and Fading Effects
class Laser(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        
        # Laser with glowing and fading effect
        self.image = pygame.Surface((5, 20), pygame.SRCALPHA)
        
        # Core of the laser
        pygame.draw.rect(self.image, GREEN, (1, 0, 3, 20))
        
        # Glowing effect with fading
        glow_surface = pygame.Surface((5, 20), pygame.SRCALPHA)
        glow_color = (0, 255, 0, 50)  # Transparent green glow
        pygame.draw.rect(glow_surface, glow_color, glow_surface.get_rect())
        self.image.blit(glow_surface, (0, 0))
        
        self.rect = self.image.get_rect()
        self.rect.centerx = x
        self.rect.bottom = y
        self.speed = -10

    def update(self):
        self.rect.y += self.speed
        # Remove laser if it goes off screen
        if self.rect.bottom < 0:
            self.kill()

# Main Game Class with Health and Bullet Increment System
class SpaceWar:
    def __init__(self):
        self.screen = screen
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 36)
        self.starfield = StarField(WIDTH, HEIGHT)
        
        # Sprite Groups
        self.all_sprites = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()
        self.lasers = pygame.sprite.Group()
        
        # Player Setup
        self.player = Player()
        self.all_sprites.add(self.player)
        
        # Game Variables
        self.score = 0

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_s and len(self.lasers) < self.player.bullet_count:
                        # Fire multiple bullets based on the player's current weapon level
                        for i in range(self.player.bullet_count):
                            offset = (i - self.player.bullet_count // 2) * 15
                            laser = Laser(self.player.rect.centerx + offset, self.player.rect.top)
                            self.all_sprites.add(laser)
                            self.lasers.add(laser)

            # Update game objects
            self.all_sprites.update()
            self.starfield.update()

            # Spawn enemies
            if random.random() < 0.02:
                enemy = Enemy()
                self.all_sprites.add(enemy)
                self.enemies.add(enemy)

            # Collision detection: Player vs Enemies
            hits = pygame.sprite.spritecollide(self.player, self.enemies, True)
            for hit in hits:
                self.player.lose_health(10)  # Decrease health on collision with an enemy
                if self.player.health <= 0:
                    running = False  # End game if health is zero

            # Collision detection: Laser vs Enemy
            hits = pygame.sprite.groupcollide(self.enemies, self.lasers, True, True)
            self.score += len(hits) * 10

            # Increase weapon level and bullets every 400 points
            if self.score >= 400 * (self.player.weapon_level):
                self.player.weapon_level += 1
                self.player.increase_bullets()  # Increase the bullet count as well

            # Draw everything
            self.screen.fill(BLACK)
            self.starfield.draw(self.screen)
            self.all_sprites.draw(self.screen)
            
            # Draw score and health
            score_text = self.font.render(f"Score: {self.score}", True, WHITE)
            self.screen.blit(score_text, (10, 10))
            
            health_text = self.font.render(f"Health: {self.player.health}", True, WHITE)
            self.screen.blit(health_text, (WIDTH - 150, 10))

            # Update display
            pygame.display.flip()

            # Control game speed
            self.clock.tick(60)

        pygame.quit()

# Run the game
def main():
    game = SpaceWar()
    game.run()

if __name__ == "__main__":
    main()
